In [1]:
from ipywidgets import interact_manual, IntText, Text
import ipywidgets as widgets
import numpy as np
from numpy.random import randint
from fractions import Fraction
import matplotlib.pyplot as plt

In [2]:
# Funkcje pomocnicze
def poly_eval(A, x): #oblicza wartość wielomianu w punkcie
    return np.polyval(A[::-1], x)

def lagr_interp(X): #oblicza współczynniki interpolacji Lagrange’a 
    result = []
    for i, xi in enumerate(X):
        num = 1
        den = 1
        for j, xj in enumerate(X):
            if i != j:
                num *= xj
                den *= xj - xi
        result.append(Fraction(num, den))
    return result

In [3]:
# ENCRYPTION
def shamir_encrypt(secret, n, t, p):
    assert p > secret, f'Liczba pierwsza p musi być większa niż sekret ({p} <= {secret})'
    assert p > n, f'Liczba pierwsza p musi być większa niż liczba udziałów ({p} <= {n})'
    assert t <= n, "Próg t musi być mniejszy lub równy liczbie udziałów n."

    # Generowanie współczynników wielomianu
    A = [secret] + list(randint(0, p, size=t-1))
    X = np.arange(1, n + 1)
    #Obliczanie wartości wielomianu dla każdego udziału (modulo p)
    Y = [poly_eval(A, x) % p for x in X]

    print("Współczynniki wielomianu:")
    for i, a in enumerate(A):
        print(f"a_{i} = {a}")

    print("\nUdziały:")
    for x, y in zip(X, Y):
        print(f"f({x}) mod p = {y}")

    print("\n--- ENCRYPTED SHARES ---")
    print(";".join(f"{x},{y}" for x, y in zip(X, Y)))

    # Wykres
    x_vals = np.linspace(0, n + 2, 200)
    y_vals = [poly_eval(A, x) % p for x in x_vals]
    plt.plot(x_vals, y_vals, label='f(x)')
    plt.scatter(X, Y, color='red', zorder=5)
    plt.title("Wielomian tajemnicy i udziały")
    plt.xlabel("x")
    plt.ylabel("f(x) mod p")
    plt.grid(True)
    plt.legend()
    plt.show()

In [4]:
# DECRYPTION
def shamir_decrypt(keys, p):
    try:
        coords = [tuple(map(int, pair.strip().split(','))) for pair in keys.strip().split(';')]
        X = np.array([c[0] for c in coords])
        Y = np.array([c[1] for c in coords])
    except Exception as e:
        print("Błąd w formacie danych wejściowych. Użyj np. 1,123;2,456;3,789")
        print(f"Szczegóły: {e}")
        return
    
    # Obliczenie współczynników interpolacyjnych Lagrange
    L = lagr_interp(X)

    S = 0
    print("Interpolacja Lagrange'a (modularna):")
    for i in range(len(X)):
        # y_i * l_i(0), gdzie l_i(0) = (iloczyn x_j) / (iloczyn (x_j - x_i)) modulo p
        num = Y[i] * L[i].numerator % p
        denom_inv = pow(int(L[i].denominator), -1, p)  # odwrotność modularna
        term = (num * denom_inv) % p
        S = (S + term) % p
        print(f"y_{i} * {L[i].numerator} / {L[i].denominator} mod {p} = {Y[i]} * {L[i].numerator} * {denom_inv} ≡ {term} (mod {p})")

    print("\n--- DECRYPTED SECRET ---")
    print(S)

In [6]:
# INTERFEJS
interact_manual(
    shamir_encrypt,
    secret=IntText(value=123, description="Sekret"),
    n=widgets.IntSlider(min=2, max=10, value=5, description="Liczba udziałów"),
    t=widgets.IntSlider(min=2, max=10, value=3, description="Próg"),
    p=IntText(value=65537, description="Liczba pierwsza")
)

interact_manual(
    shamir_decrypt,
    keys=Text(value="1,123;2,456;3,789", description="Udziały"),
    p=IntText(value=65537, description="Liczba pierwsza")
)

interactive(children=(IntText(value=123, description='Sekret'), IntSlider(value=5, description='Liczba udziałó…

interactive(children=(Text(value='1,123;2,456;3,789', description='Udziały'), IntText(value=65537, description…

<function __main__.shamir_decrypt(keys, p)>